In [8]:
import requests
import pandas as pd
import datetime
import pymysql
import sqlalchemy
import numpy as np
import keys
from keys import *

# Airport Arrivals

Find the Endpoint that contains the desired information to collect data about flight arrivals to an airport. If you need the IATA or ICAO codes of an airport, you can use an endpoint from this same API… or you can just Google it.

Every day, the company wants to know which flights will arrive the next day. To automate this, you will have to come up with code that, when executed, generates tomorrow’s date and transforms it into the format that that particular API endpoint requires.

If you feel like watching a video walkthrough about what APIs are and how to interact with them, you can hop on to the fourth chapter of this course:

In [9]:
url = "https://aerodatabox.p.rapidapi.com/airports/icao/EDDP"

headers = {
	"X-RapidAPI-Key": aerodatabox_key,
	"X-RapidAPI-Host": "aerodatabox.p.rapidapi.com"
}

response = requests.request("GET", url, headers=headers)

print(response.text)

response.json()

{"icao":"EDDP","iata":"LEJ","shortName":"Halle","fullName":"Leipzig, Leipzig Halle","municipalityName":"Leipzig","location":{"lat":51.4324,"lon":12.2416},"country":{"code":"DE","name":"Germany"},"continent":{"code":"EU","name":"Europe"},"timeZone":"Europe/Berlin","urls":{"webSite":"https://www.leipzig-halle-airport.de","wikipedia":"https://en.wikipedia.org/wiki/Leipzig/Halle_Airport","twitter":"http://twitter.com/lej_airport","googleMaps":"https://www.google.com/maps/@51.432399,12.241600,14z","flightRadar":"https://www.flightradar24.com/51.43,12.24/14"}}


{'icao': 'EDDP',
 'iata': 'LEJ',
 'shortName': 'Halle',
 'fullName': 'Leipzig, Leipzig Halle',
 'municipalityName': 'Leipzig',
 'location': {'lat': 51.4324, 'lon': 12.2416},
 'country': {'code': 'DE', 'name': 'Germany'},
 'continent': {'code': 'EU', 'name': 'Europe'},
 'timeZone': 'Europe/Berlin',
 'urls': {'webSite': 'https://www.leipzig-halle-airport.de',
  'wikipedia': 'https://en.wikipedia.org/wiki/Leipzig/Halle_Airport',
  'twitter': 'http://twitter.com/lej_airport',
  'googleMaps': 'https://www.google.com/maps/@51.432399,12.241600,14z',
  'flightRadar': 'https://www.flightradar24.com/51.43,12.24/14'}}

Generating tomorrow'sdate

In [10]:
tommorrow=datetime.date.today()+ datetime.timedelta(days=1)
tomrowstr=str(tommorrow)
tomrowstr

'2022-08-11'

Creating a function, that returns all flights departured on the next day from 8 am to 7pm at the chosen airport.Since IATA is not so commmonly used like, we use an additional API call to extract the needed ICAO Code for this airport.

In [11]:

def departured_flights8to12(iata):
    url_1 = "https://aerodatabox.p.rapidapi.com/airports/iata/"+iata+""

    headers = {
        "X-RapidAPI-Key": aerodatabox_key,
        "X-RapidAPI-Host": "aerodatabox.p.rapidapi.com"
    }

    response_1 = requests.request("GET", url_1, headers=headers)    
    icaoc=response_1.json()["icao"]
    
    json_object=response_1.json()["icao"]
    
    url = "https://aerodatabox.p.rapidapi.com/flights/airports/icao/"+icaoc+"/"+tomrowstr+"T08:00/"+tomrowstr+"T12:00"

    querystring = {"withLeg":"true","withCancelled":"true","withCodeshared":"true","withCargo":"true","withPrivate":"true","withLocation":"false"}

    headers = {
        "X-RapidAPI-Key": aerodatabox_key,
        "X-RapiddAPI-Host": "aerodatabox.p.rapidapi.com"
    }

    response = requests.request("GET", url, headers=headers, params=querystring)
    data=response.json()["departures"]
    #print(response.text)
    flights=[]
    for i in data:
        flights_dic={}
        
        if "scheduledTimeLocal" in i["departure"].keys():
            flights_dic["Departure Local Time"]=i["departure"]["scheduledTimeLocal"]
        else:
            flights_dic["Departure Local Time"]="unknown"
            
    #    flights_dic["Departure Local Time"]=i["departure"]["scheduledTimeLocal"]
        
        if "scheduledTimeLocal" in i["arrival"].keys():
            flights_dic["Arriving at (Local Time)"]=i["arrival"]["scheduledTimeLocal"]
        else:
            flights_dic["Arriving at (Local Time)"]="unknown"
        #flights_dic["Arriving at (Local Time)"]=i["arrival"]["scheduledTimeLocal"]
        if "airport" in i["arrival"].keys():
            flights_dic["To"]=i["arrival"]["airport"]["name"]
        else:
            flights_dic["To"]="unknown" 
            
        if "number" in i.keys():
            flights_dic["Flight number"]=i["number"]
        else:
            flights_dic["Flight number"]="unknown"        
        if "name" in i["airline"].keys():
            flights_dic["Airline"]=i["airline"]["name"]
        else:
            flights_dic["Airline"]="unknown"         
        if "aircraft" in i.keys():
            flights_dic["Aircraft"]=i["aircraft"]["model"]
        else:
            flights_dic["Aircraft"]="unknown"

        #flights_dic["Aircraft"]=i["aircraft"]["model"]
        flights.append(flights_dic)
        df_flights=pd.DataFrame(flights)
    return df_flights
    #df_flights

In [12]:
departured_flights8to12("MUC")

,Departure Local Time,Arriving at (Local Time),To,Flight number,Airline,Aircraft
0,2022-08-11 09:45+02:00,unknown,Heraklion,4Y 1256,Airbus Transport International,unknown
1,2022-08-11 10:20+02:00,unknown,Antalya,XC 4104,Corendon Air,Airbus A320
2,2022-08-11 10:05+02:00,unknown,Ankara,XC 5046,Corendon Air,unknown
3,2022-08-11 11:50+02:00,unknown,Toronto,AC 835,Air Canada,Boeing 777-200LR
4,2022-08-11 11:00+02:00,unknown,Arbil,UD 442,UR,unknown
...,...,...,...,...,...,...
131,2022-08-11 10:25+02:00,2022-08-11 14:10+03:00,Bodrum,XC 6944,Corendon Air,Airbus A320
132,2022-08-11 10:50+02:00,2022-08-11 14:30+03:00,Istanbul,TK 1630,Turkish,Airbus A330-200
133,2022-08-11 11:00+02:00,2022-08-11 16:15+03:00,Arbil,W2 2242,FlexFlight,Airbus A320
134,2022-08-11 09:40+02:00,2022-08-11 16:20+03:00,Doha,QR 60,Qatar Airways,Boeing 787-8


Based on information how the data is structured, we build a function, that provides us all arriving flights from 8 am to 7pm.

In [13]:
def arflights8to19(iata):
    
    url_1 = "https://aerodatabox.p.rapidapi.com/airports/iata/"+iata+""

    headers = {
        "X-RapidAPI-Key": aerodatabox_key,
        "X-RapidAPI-Host": "aerodatabox.p.rapidapi.com"
    }

    response_1 = requests.request("GET", url_1, headers=headers)    
    icaoc=response_1.json()["icao"]
    
    json_object=response_1.json()["icao"]
    
    url = "https://aerodatabox.p.rapidapi.com/flights/airports/icao/"+icaoc+"/"+tomrowstr+"T08:00/"+tomrowstr+"T19:00"

    querystring = {"withLeg":"true","withCancelled":"true","withCodeshared":"true","withCargo":"true","withPrivate":"true","withLocation":"false"}

    headers = {
        "X-RapidAPI-Key": aerodatabox_key,
        "X-RapidAPI-Host": "aerodatabox.p.rapidapi.com"
    }

    response = requests.request("GET", url, headers=headers, params=querystring)
    data=response.json()["arrivals"]
    #print(response.text)
    flights=[]
    
    for i in data:
        flights_dic={}
        if "airport" in i["departure"].keys():#departure from
            flights_dic["departure_from"]=i["departure"]["airport"]["name"] 
        else:
            flights_dic["departure_from"]="unknown"
        if "scheduledTimeLocal" in i["departure"].keys():
            flights_dic["departure_time"]=i["departure"]["scheduledTimeLocal"]#departure time local
        else:
            flights_dic["departure_time"]="unknown"
            
        if "scheduledTimeLocal" in i["arrival"].keys():
            flights_dic["arrival_time"]=i["arrival"]["scheduledTimeLocal"]#arrival time local
        else:
            flights_dic["arrival_time"]="unknown"
            
        if "number" in i.keys():
            flights_dic["flight_number"]=i["number"] #flight number
        else:
            flights_dic["flight_number"]="unknown"
            
        if "terminal" in i["arrival"].keys():#departure from
            flights_dic["terminal"]=i["arrival"]["terminal"]
        else:
            flights_dic["terminal"]="unknown"
            
        if "airline" in i.keys():
            flights_dic["airline"]=i["airline"]["name"] #airline
        else:
            flights_dic["airline"]="unknown"
            
        if "aircraft" in i.keys(): #aicraft
            flights_dic["aircraft"]=i["aircraft"]["model"]
        else:
            flights_dic["aircraft"]="unknown"
        if "icao" in i["departure"]["airport"].keys():
            flights_dic["ICAO"]=i["departure"]["airport"]["icao"]  #icao
        else:
            flights_dic["ICAO"]="unknown"            

        if "iata" in i["departure"]["airport"].keys():
            flights_dic["IATA"]=i["departure"]["airport"]["iata"] #iata
        else:
            flights_dic["IATA"]="unknown"           

        flights.append(flights_dic)
        df_flights=pd.DataFrame(flights)
        df_flights["departure_time"]=df_flights["departure_time"].str.extract("(\d\d\d\d-\d\d-\d\d \d\d:\d\d)")
        df_flights["arrival_time"]=df_flights["arrival_time"].str.extract("(\d\d\d\d-\d\d-\d\d \d\d:\d\d)")
        df_flights["departure_time"]=pd.to_datetime(df_flights["departure_time"])
        df_flights["arrival_time"]=pd.to_datetime(df_flights["arrival_time"])
        df_flights = df_flights[df_flights["ICAO"] != "unknown"]
    return df_flights

In [14]:
df_arrivals=arflights8to19("BER")
df_arrivals

,departure_from,departure_time,arrival_time,flight_number,terminal,airline,aircraft,ICAO,IATA
0,Helsinki,2022-08-11 07:25:00,2022-08-11 08:25:00,AY 1431,1,Finnair,Airbus A320,EFHK,HEL
1,Riga,2022-08-11 07:45:00,2022-08-11 08:20:00,BT 211,1,Air Baltic,Fokker 100,EVRA,RIX
2,Oslo,2022-08-11 07:00:00,2022-08-11 08:35:00,DY 1100,1,Norwegian Air Shuttle,Boeing 737-800,ENGM,OSL
3,Budapest,2022-08-11 06:55:00,2022-08-11 08:25:00,FR 197,0,Ryanair,Boeing 737-800,LHBP,BUD
4,Treviso,2022-08-11 07:10:00,2022-08-11 08:40:00,FR 2607,0,Ryanair,Boeing 737-800,LIPH,TSF
...,...,...,...,...,...,...,...,...,...
137,Antalya,2022-08-11 15:25:00,2022-08-11 18:00:00,PC 5007,0,Pegasus,Airbus A320,LTAI,AYT
138,Istanbul,2022-08-11 16:40:00,2022-08-11 18:40:00,PC 983,0,Pegasus,Boeing 737-800,LTFJ,SAW
139,Kerkyra Island,2022-08-11 16:40:00,2022-08-11 18:00:00,U2 4732,1,easyJet,Airbus A320,LGKR,CFU
140,Milan,2022-08-11 16:55:00,2022-08-11 18:45:00,U2 5186,1,easyJet,Airbus A319,LIML,LIN


In [15]:
df_arrivals.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 141 entries, 0 to 141
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   departure_from  141 non-null    object        
 1   departure_time  141 non-null    datetime64[ns]
 2   arrival_time    141 non-null    datetime64[ns]
 3   flight_number   141 non-null    object        
 4   terminal        141 non-null    object        
 5   airline         141 non-null    object        
 6   aircraft        141 non-null    object        
 7   ICAO            141 non-null    object        
 8   IATA            141 non-null    object        
dtypes: datetime64[ns](2), object(7)
memory usage: 11.0+ KB


# Open Weather

Weather is a huge factor when it comes to scooter usage. On very cold days and, especially, on rainy days, hardly anyone wants to use a scooter.

Thankfully, there is an API that provides free weather forecasts: OpenWeather.
Obviously, city and API_key are variables that you will have already defined. Once you’ve made this API call, you’ll be able to explore the received information using the requests library and pandas.

The goal is to automate the collection of weather data for multiple cities.

Explore the data received from the API. What’s useful? What’s not?
Extract the information you see as useful and make a DataFrame from it
Consolidate the steps you took to make the DataFrame into a single function. The function should output a DataFrame with the weather of multiple cities, when given a list of cities as an input.
You should be able to use this function to get the weather data for the cities you web scraped yesterday

In [16]:
city="Berlin"
response_1 = requests.get('http://api.openweathermap.org/data/2.5/forecast?q='+city+'&appid='+owmkey+'&units=metric"')
response_berlin = response_1.json() #formats the scraped data into JSON format
response_berlin #berlin weather forecasts for 5 days every 3 hours as JSON

{'cod': '200',
 'message': 0,
 'cnt': 40,
 'list': [{'dt': 1660132800,
   'main': {'temp': 298.3,
    'feels_like': 297.97,
    'temp_min': 298.3,
    'temp_max': 300.61,
    'pressure': 1026,
    'sea_level': 1026,
    'grnd_level': 1021,
    'humidity': 42,
    'temp_kf': -2.31},
   'weather': [{'id': 802,
     'main': 'Clouds',
     'description': 'scattered clouds',
     'icon': '03d'}],
   'clouds': {'all': 25},
   'wind': {'speed': 1.62, 'deg': 29, 'gust': 1.91},
   'visibility': 10000,
   'pop': 0,
   'sys': {'pod': 'd'},
   'dt_txt': '2022-08-10 12:00:00'},
  {'dt': 1660143600,
   'main': {'temp': 300.05,
    'feels_like': 299.63,
    'temp_min': 300.05,
    'temp_max': 301.5,
    'pressure': 1025,
    'sea_level': 1025,
    'grnd_level': 1019,
    'humidity': 33,
    'temp_kf': -1.45},
   'weather': [{'id': 802,
     'main': 'Clouds',
     'description': 'scattered clouds',
     'icon': '03d'}],
   'clouds': {'all': 25},
   'wind': {'speed': 1.63, 'deg': 25, 'gust': 2.48},
   

In [17]:
response_berlin["list"][6]["weather"][0]["description"]

'few clouds'

In [18]:
temps_berlin = []
for i in response_berlin["list"]:
    temps_dict={}
    temps_dict["Weather"]=i["weather"][0]["description"]
    temps_dict['Temparature'] = i['main']["temp"]
    temps_dict['Feels like'] = i['main']["feels_like"]
    temps_dict['Humidity'] = i['main']["humidity"]
    temps_dict["Timestamp"]= i["dt_txt"]
    temps_berlin.append(temps_dict)
    df_ber_temp=pd.DataFrame(temps_berlin)

In [19]:
df_ber_temp

,Weather,Temparature,Feels like,Humidity,Timestamp
0,scattered clouds,298.30,297.97,42,2022-08-10 12:00:00
1,scattered clouds,300.05,299.63,33,2022-08-10 15:00:00
2,scattered clouds,298.84,298.33,33,2022-08-10 18:00:00
3,clear sky,293.89,293.12,42,2022-08-10 21:00:00
4,clear sky,291.56,290.74,49,2022-08-11 00:00:00
5,clear sky,290.56,289.64,49,2022-08-11 03:00:00
6,few clouds,293.28,292.47,43,2022-08-11 06:00:00
7,few clouds,299.37,299.37,30,2022-08-11 09:00:00
8,few clouds,302.65,301.05,22,2022-08-11 12:00:00
9,scattered clouds,303.50,301.64,19,2022-08-11 15:00:00


In [20]:
def temps(city, country):
    temps_city = []
    response = requests.get(f'http://api.openweathermap.org/data/2.5/forecast/?q='+city+','+country+'&appid='+owmkey+'&units=metric&lang=en')
    response_city = response.json()["list"]
    for i in response_city:
        temps_dict={}
        
        temps_dict["weather"]=i["weather"][0]["description"]
        temps_dict['temparature'] = i['main']["temp"]
        temps_dict['feels_like'] = i['main']["feels_like"]
        temps_dict['humidity'] = i['main']["humidity"]
        temps_dict["timestamp"]= i["dt_txt"]
        temps_dict['windspeed']=i["wind"]["speed"]
        temps_dict['municipality_iso_country']=city+","+country
        #temps_dict["municipality_iso_country"]=np.where(city cities["city_ascii"]==,cities["municipality_iso_country"], "AAA" )
        temps_city.append(temps_dict)
        df_city_temp=pd.DataFrame(temps_city)
        #city_forecast=df_city_temp.merge(cities, how="left", left_on="city", right_on="city_ascii"   
        df_city_temp["timestamp"]=pd.to_datetime(df_city_temp["timestamp"])
    return df_city_temp  

In [21]:
df_temps_in_berlin=pd.DataFrame(temps_berlin)
df_temps_in_berlin

,Weather,Temparature,Feels like,Humidity,Timestamp
0,scattered clouds,298.30,297.97,42,2022-08-10 12:00:00
1,scattered clouds,300.05,299.63,33,2022-08-10 15:00:00
2,scattered clouds,298.84,298.33,33,2022-08-10 18:00:00
3,clear sky,293.89,293.12,42,2022-08-10 21:00:00
4,clear sky,291.56,290.74,49,2022-08-11 00:00:00
5,clear sky,290.56,289.64,49,2022-08-11 03:00:00
6,few clouds,293.28,292.47,43,2022-08-11 06:00:00
7,few clouds,299.37,299.37,30,2022-08-11 09:00:00
8,few clouds,302.65,301.05,22,2022-08-11 12:00:00
9,scattered clouds,303.50,301.64,19,2022-08-11 15:00:00


# Airport Data

To get information about all airports we simply load a csv with all airports, found here(https://ourairports.com/data/), into this notebook. Then we filter and adding all columns and rows we need. 

In [22]:
#filtered_df = df.query( '"Country Name" == ["brazil","poland","russia","countrydummy","usa"]' )

airports_cities = (
pd.read_csv('airports.csv')
    .query('type == ["large_airport","medium_airport"]')
    .filter(['name','type','latitude_deg','longitude_deg','iso_country','iso_region','municipality','gps_code','iata_code'])
    .rename(columns={'gps_code':'icao_code'})
    .assign(municipality_iso_country = lambda x: x['municipality'] + ',' + x['iso_country'])
)
airports_cities.head()
airports_cities=airports_cities.drop_duplicates(subset="icao_code")


In [23]:
airports_cities

,name,type,latitude_deg,longitude_deg,iso_country,iso_region,municipality,icao_code,iata_code,municipality_iso_country
6401,Aleknagik / New Airport,medium_airport,59.282600,-158.617996,US,US-AK,Aleknagik,5A8,WKK,"Aleknagik,US"
10856,Khost International Airport (U.C.),medium_airport,33.284605,69.807340,AF,AF-KHO,Khost,NaN,NaN,"Khost,AF"
10890,Honiara International Airport,large_airport,-9.428000,160.054993,SB,SB-CT,Honiara,AGGH,HIR,"Honiara,SB"
10895,Munda Airport,medium_airport,-8.327970,157.263000,SB,SB-WE,Munda,AGGM,MUA,"Munda,SB"
10917,Hongyuan Airport,medium_airport,32.531540,102.352240,CN,CN-51,Aba,ZUHY,AHJ,"Aba,CN"
...,...,...,...,...,...,...,...,...,...,...
68322,Tonghua Sanyuanpu Airport,medium_airport,42.253889,125.703333,CN,CN-22,Tonghua,ZYTN,TNH,"Tonghua,CN"
68323,Shenyang Taoxian International Airport,large_airport,41.639801,123.483002,CN,CN-21,"Hunnan, Shenyang",ZYTX,SHE,"Hunnan, Shenyang,CN"
68325,Yanji Chaoyangchuan Airport,medium_airport,42.882801,129.451004,CN,CN-22,Yanji,ZYYJ,YNJ,"Yanji,CN"
68326,Yingkou Lanqi Airport,medium_airport,40.542524,122.358600,CN,CN-21,"Laobian, Yingkou",ZYYK,YKH,"Laobian, Yingkou,CN"


Then we drop all NaNs...

In [24]:
airports_cities=airports_cities.dropna()

... and check if there any other irregularities.

In [25]:
airports_cities.isna().sum()

name                        0
type                        0
latitude_deg                0
longitude_deg               0
iso_country                 0
iso_region                  0
municipality                0
icao_code                   0
iata_code                   0
municipality_iso_country    0
dtype: int64

# Population Data

To get information about all cities we simply load a csv, provided with additional information, found here(https://www.kaggle.com/datasets/juanmah/world-cities), into this notebook. Then we filter and adding all columns we need.

In [26]:
cities = (
pd.read_csv('worldcities.csv')
)


Doing some data manipulation, in order to make it consistent with the data frame 'airport_cities'.

In [27]:
cities.at[1002,'city_ascii']='Frankfurt-am-Main'
cities.query("city_ascii=='Frankfurt-am-Main'")

,city,city_ascii,lat,lng,country,iso2,iso3,admin_name,capital,population,id
1002,Frankfurt,Frankfurt-am-Main,50.1136,8.6797,Germany,DE,DEU,Hesse,minor,764104.0,1276054552


Creating this column, in order to have relation to 'airport_cities'.

In [28]:
cities["municipality_iso_country"]=cities["city_ascii"]+","+cities["iso2"]
cities.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42905 entries, 0 to 42904
Data columns (total 12 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   city                      42905 non-null  object 
 1   city_ascii                42905 non-null  object 
 2   lat                       42905 non-null  float64
 3   lng                       42905 non-null  float64
 4   country                   42905 non-null  object 
 5   iso2                      42873 non-null  object 
 6   iso3                      42905 non-null  object 
 7   admin_name                42810 non-null  object 
 8   capital                   9812 non-null   object 
 9   population                42180 non-null  float64
 10  id                        42905 non-null  int64  
 11  municipality_iso_country  42873 non-null  object 
dtypes: float64(3), int64(1), object(8)
memory usage: 3.9+ MB


Removing rows with irregular data, in order to have consistent data.

In [29]:
#cities = cities.filter(['city_ascii','iso_country','municipality_iso_country']).drop_duplicates()
cities=cities.dropna()

In [30]:
cities.isna().sum()

city                        0
city_ascii                  0
lat                         0
lng                         0
country                     0
iso2                        0
iso3                        0
admin_name                  0
capital                     0
population                  0
id                          0
municipality_iso_country    0
dtype: int64

Filtering all cities with a larger population than 1000 to make this dataset a little bit more compact, since we assume ,that all cities with population smaller than 1000 doesnt have a medium or small airport.

In [31]:
cities=cities.sort_values("population", ascending= False)


i = cities[(cities.population<1000 )].index
cities=cities.drop(i)
cities

,city,city_ascii,lat,lng,country,iso2,iso3,admin_name,capital,population,id,municipality_iso_country
0,Tokyo,Tokyo,35.6839,139.7744,Japan,JP,JPN,Tōkyō,primary,39105000.0,1392685764,"Tokyo,JP"
1,Jakarta,Jakarta,-6.2146,106.8451,Indonesia,ID,IDN,Jakarta,primary,35362000.0,1360771077,"Jakarta,ID"
2,Delhi,Delhi,28.6667,77.2167,India,IN,IND,Delhi,admin,31870000.0,1356872604,"Delhi,IN"
3,Manila,Manila,14.6000,120.9833,Philippines,PH,PHL,Manila,primary,23971000.0,1608618140,"Manila,PH"
4,São Paulo,Sao Paulo,-23.5504,-46.6339,Brazil,BR,BRA,São Paulo,admin,22495000.0,1076532519,"Sao Paulo,BR"
...,...,...,...,...,...,...,...,...,...,...,...,...
9461,Kobarid,Kobarid,46.2471,13.5796,Slovenia,SI,SVN,Kobarid,admin,1085.0,1705651680,"Kobarid,SI"
9463,Andrijevica,Andrijevica,42.7339,19.7919,Montenegro,ME,MNE,Andrijevica,admin,1073.0,1499379378,"Andrijevica,ME"
9454,Bosilovo,Bosilovo,41.4406,22.7278,Macedonia,MK,MKD,Bosilovo,admin,1069.0,1807868218,"Bosilovo,MK"
42150,Mazatán,Mazatan,29.0167,-110.1333,Mexico,MX,MEX,Sonora,minor,1058.0,1484350756,"Mazatan,MX"


Resetting the index, after we our final data manipulation.

In [32]:
cities=cities.drop_duplicates(subset=['municipality_iso_country'])
cities=cities.reset_index()

In [33]:
del cities["index"]

In [34]:
cities

,city,city_ascii,lat,lng,country,iso2,iso3,admin_name,capital,population,id,municipality_iso_country
0,Tokyo,Tokyo,35.6839,139.7744,Japan,JP,JPN,Tōkyō,primary,39105000.0,1392685764,"Tokyo,JP"
1,Jakarta,Jakarta,-6.2146,106.8451,Indonesia,ID,IDN,Jakarta,primary,35362000.0,1360771077,"Jakarta,ID"
2,Delhi,Delhi,28.6667,77.2167,India,IN,IND,Delhi,admin,31870000.0,1356872604,"Delhi,IN"
3,Manila,Manila,14.6000,120.9833,Philippines,PH,PHL,Manila,primary,23971000.0,1608618140,"Manila,PH"
4,São Paulo,Sao Paulo,-23.5504,-46.6339,Brazil,BR,BRA,São Paulo,admin,22495000.0,1076532519,"Sao Paulo,BR"
...,...,...,...,...,...,...,...,...,...,...,...,...
8904,Kobarid,Kobarid,46.2471,13.5796,Slovenia,SI,SVN,Kobarid,admin,1085.0,1705651680,"Kobarid,SI"
8905,Andrijevica,Andrijevica,42.7339,19.7919,Montenegro,ME,MNE,Andrijevica,admin,1073.0,1499379378,"Andrijevica,ME"
8906,Bosilovo,Bosilovo,41.4406,22.7278,Macedonia,MK,MKD,Bosilovo,admin,1069.0,1807868218,"Bosilovo,MK"
8907,Mazatán,Mazatan,29.0167,-110.1333,Mexico,MX,MEX,Sonora,minor,1058.0,1484350756,"Mazatan,MX"


Renaming some the last two columns

In [35]:
cities=cities.rename(columns={"lng": "longitude"}, errors="raise")
cities=cities.rename(columns={"lat": "latitude"}, errors="raise")



In [36]:
cities.query("municipality_iso_country=='Berlin,DE'")


,city,city_ascii,latitude,longitude,country,iso2,iso3,admin_name,capital,population,id,municipality_iso_country
152,Berlin,Berlin,52.5167,13.3833,Germany,DE,DEU,Berlin,primary,3664088.0,1276451290,"Berlin,DE"


# Check data

In [43]:
cities.info()
airports_cities.info()
df_arrivals.info()
df_temps_in_berlin.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8909 entries, 0 to 8908
Data columns (total 12 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   city                      8909 non-null   object 
 1   city_ascii                8909 non-null   object 
 2   latitude                  8909 non-null   float64
 3   longitude                 8909 non-null   float64
 4   country                   8909 non-null   object 
 5   iso2                      8909 non-null   object 
 6   iso3                      8909 non-null   object 
 7   admin_name                8909 non-null   object 
 8   capital                   8909 non-null   object 
 9   population                8909 non-null   float64
 10  id                        8909 non-null   int64  
 11  municipality_iso_country  8909 non-null   object 
dtypes: float64(3), int64(1), object(8)
memory usage: 835.3+ KB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 4216 entr

We are merging 'airport_cities' and 'cities' in order to detect airport cities, which are not in 'cities' and avoid errors, when we late define the foreign keys.

In [44]:
airport_cities=airports_cities.merge(cities, left_on="municipality_iso_country", right_on="municipality_iso_country")

In [45]:
airport_cities.isna().sum()

name                        0
type                        0
latitude_deg                0
longitude_deg               0
iso_country                 0
iso_region                  0
municipality                0
icao_code                   0
iata_code                   0
municipality_iso_country    0
city                        0
city_ascii                  0
latitude                    0
longitude                   0
country                     0
iso2                        0
iso3                        0
admin_name                  0
capital                     0
population                  0
id                          0
dtype: int64

Deleting unnecessary columns

In [46]:
airport_cities

del airport_cities["city_ascii"]
del airport_cities["country"]
del airport_cities["iso2"]
del airport_cities["iso3"]
del airport_cities["id"]
del airport_cities["population"]
del airport_cities["city"]
del airport_cities["admin_name"]
del airport_cities["capital"]


In [47]:
#del airport_cities["city"]
airport_cities.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 1341 entries, 0 to 1340
Data columns (total 12 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   name                      1341 non-null   object 
 1   type                      1341 non-null   object 
 2   latitude_deg              1341 non-null   float64
 3   longitude_deg             1341 non-null   float64
 4   iso_country               1341 non-null   object 
 5   iso_region                1341 non-null   object 
 6   municipality              1341 non-null   object 
 7   icao_code                 1341 non-null   object 
 8   iata_code                 1341 non-null   object 
 9   municipality_iso_country  1341 non-null   object 
 10  latitude                  1341 non-null   float64
 11  longitude                 1341 non-null   float64
dtypes: float64(4), object(8)
memory usage: 136.2+ KB


We are merging 'df_arrivals' and 'airport_cities' in order to detect airports , which are not in 'airport_cities' and avoid errors, when we late define the foreign keys.

In [52]:
df_arrivals.merge(airports_cities, how="left", left_on="ICAO", right_on="icao_code").isna().sum()

departure_from              0
departure_time              0
arrival_time                0
flight_number               0
terminal                    0
airline                     0
aircraft                    0
ICAO                        0
IATA                        0
name                        0
type                        0
latitude_deg                0
longitude_deg               0
iso_country                 0
iso_region                  0
municipality                0
icao_code                   0
iata_code                   0
municipality_iso_country    0
dtype: int64

In [53]:
df_arrivals.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 141 entries, 0 to 141
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   departure_from  141 non-null    object        
 1   departure_time  141 non-null    datetime64[ns]
 2   arrival_time    141 non-null    datetime64[ns]
 3   flight_number   141 non-null    object        
 4   terminal        141 non-null    object        
 5   airline         141 non-null    object        
 6   aircraft        141 non-null    object        
 7   ICAO            141 non-null    object        
 8   IATA            141 non-null    object        
dtypes: datetime64[ns](2), object(7)
memory usage: 11.0+ KB


# SQL

We are now trying to establish a local MySQL Connection...

In [185]:
schema="gans"
host="127.0.0.1"
user="root"
password=loc_pw
port=3306
con = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

... and after creating the database and tables in MySQL we fill up those tables with the data from the data frames

In [221]:
cities.to_sql('cities', con=con, if_exists='append', index=False)



8909

In [232]:
airport_cities.to_sql('airport_cities', if_exists='append', con=con, index=False)

1341

In [ ]:
df_arrivals.to_sql('arrivals', if_exists='append', con=con, index=False)


In [ ]:
df_temps_in_berlin.to_sql('weather', if_exists='append', con=con, index=False)


# Connecting to Amazon RDS

Now after this connection succedded, we are now trying to establish a MySQL Connection to the Relational Database created in AWS via RDS.

In [ ]:
aaa=temps("Hannover", "DE")
aaa.info()

In [ ]:
schema="gans"
host=rds_host
user="admin" 
password=rds_pw
port=3306
con = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

In [ ]:
aaa.to_sql('weather', if_exists='append', con=con, index=False)

In [221]:
cities.to_sql('cities', con=con, if_exists='append', index=False)



8909

In [232]:
airport_cities.to_sql('airport_cities', if_exists='append', con=con, index=False)

1341

In [ ]:
df_arrivals.to_sql('arrivals', if_exists='append', con=con, index=False)


After this is finished succeded as well, we can automatically update the data from 'df_ber_temp' and 'df_arrivals'. 

To do this, we first set up lambda functions via AWS to make the API calls.

Then, we set up a timed trigger for these functions that calls these functions at timed intervals.(The Lambda functions are in 'Lambda F1.py' and 'Lambda F2.py', but pretty useless out the this AWS environment)